### 필요 라이브러리 import 및 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

data = pd.read_csv('label_1108.csv')

### icls_cnts 컬럼 정규화

In [2]:
import re 

### 원활한 vectorizing을 위해 icls_cnts 정규화
### 단, 공백으로 구분해주지 않을 경우 단어를 인식하지 못하므로 공백으로 구분
def text_cleaning(text):
    hangul = re.compile('[^ㄱ-ㅣ가-힣]+')
    result = hangul.sub(' ',text)
    return result

In [3]:
data['ko_text'] = data['icls_cnts'].apply(lambda x: text_cleaning(x))
data.head()

,Unnamed: 0,icls_cnts,label,ko_text
0,0,..<b>경찰</b>과 추격전까지 벌인 40대 남성이 붙잡혔습니다. .. 1시간이 ...,1.0,경찰 과 추격전까지 벌인 대 남성이 붙잡혔습니다 시간이 넘도록 음주운전 차량을 뒤...
1,3,"수원남부署, 모범<b>경찰</b> 최보라·김세홍 경장에 표창·장려장 수원남부<b>경...",1.0,수원남부 모범 경찰 최보라 김세홍 경장에 표창 장려장 수원남부 경찰 서 서장 오문교...
2,4,조두순에 대한 1대1 전자감독 및 <b>경찰</b>을 통한 24시간 밀착 감독 등...,1.0,조두순에 대한 대 전자감독 및 경찰 을 통한 시간 밀착 감독 등이 주된 내용이다 ...
3,5,"순천<b>경찰</b>서, 전화금융사기 예방 홍보시스템 구축 순천<b>경찰</b>서는...",1.0,순천 경찰 서 전화금융사기 예방 홍보시스템 구축 순천 경찰 서는 지난 일 전화금융사...
4,6,"경남<b>경찰</b>청 직원들, 사랑의 헌혈 릴레이 운동 경남지방<b>경찰</b>청...",1.0,경남 경찰 청 직원들 사랑의 헌혈 릴레이 운동 경남지방 경찰 청 치안감 남구준 은 ...


In [4]:
data1 = data[['ko_text','label']]
data2 = data[['ko_text','label']]

In [5]:
data1['label'].replace(-1, 1, inplace = True)
trash = data2['label'].isin([0])
data2 = data2[~trash]

/tmp/ipykernel_42762/2362880735.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['label'].replace(-1, 1, inplace = True)


In [6]:
data2

,ko_text,label
0,경찰 과 추격전까지 벌인 대 남성이 붙잡혔습니다 시간이 넘도록 음주운전 차량을 뒤...,1.0
1,수원남부 모범 경찰 최보라 김세홍 경장에 표창 장려장 수원남부 경찰 서 서장 오문교...,1.0
2,조두순에 대한 대 전자감독 및 경찰 을 통한 시간 밀착 감독 등이 주된 내용이다 ...,1.0
3,순천 경찰 서 전화금융사기 예방 홍보시스템 구축 순천 경찰 서는 지난 일 전화금융사...,1.0
4,경남 경찰 청 직원들 사랑의 헌혈 릴레이 운동 경남지방 경찰 청 치안감 남구준 은 ...,1.0
...,...,...
8054,경찰 현장 대응력 강화 태스크포스 는 차 회의에서 이 같은 내용을 결정했다 시범운...,1.0
8058,차에 불지르고 흉기 난동 대 테이저건으로 제압 대구에서 흉기로 행인을 위협하고 남의...,1.0
8064,사설 범인 제압 훈련을 온라인으로 때웠다는 경찰 대통령의 질타와 경찰청장의 사과에...,-1.0
8065,울산 경찰 방역수칙 위반 유흥주점 업주와 손님 등 명 입건 울산 경찰 청은 신종 코...,1.0


# 단순 분류 모델 

- 1차 : 중립 - not 중립 구분

- 2차 : 긍정 - 부정 구분

### 중립 - not 중립 구분

In [7]:
# 분석 변수
news1 = list(data1['ko_text'])

# 타겟 변수
y1 = np.array(data1['label'])

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = 'word', max_features = 7000)

news1 = vectorizer.fit_transform(news1)

In [9]:
from sklearn.model_selection import train_test_split

Test_size = 0.2
Random_seed = 34

train_x1, test_x1, train_y1, test_y1 = train_test_split(news1, y1, test_size = Test_size, random_state = Random_seed)

In [10]:
from sklearn.ensemble import RandomForestClassifier

model1 = RandomForestClassifier(n_estimators = 280)

model1.fit(train_x1, train_y1)

RandomForestClassifier(n_estimators=280)

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, roc_curve


y_pred1 = model1.predict(test_x1)
y_pred_proba1 = model1.predict_proba(test_x1)
roc_auc1 = roc_auc_score(test_y1, y_pred1)

print("정확도:%.2f"% accuracy_score(test_y1, y_pred1))
print("정밀도:%.3f"% precision_score(test_y1, y_pred1, average = 'weighted'))
print("재현율:%.3f"% recall_score(test_y1, y_pred1, average = 'weighted'))
print("F1:%.3f"% f1_score(test_y1, y_pred1, average = 'weighted'))
print("ROC-AUC Score : {}".format(roc_auc1))

정확도:0.81
정밀도:0.815
재현율:0.812
F1:0.802
ROC-AUC Score : 0.7589743331046451


### 긍정 - 부정 구분

In [12]:
# 분석 변수
news2 = list(data2['ko_text'])

# 타겟 변수
y2 = np.array(data2['label'])

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = 'word', max_features = 7000)

news2 = vectorizer.fit_transform(news2)

In [14]:
from sklearn.model_selection import train_test_split

Test_size = 0.2
Random_seed = 34

train_x2, test_x2, train_y2, test_y2 = train_test_split(news2, y2, test_size = Test_size, random_state = Random_seed)

In [15]:
from sklearn.ensemble import RandomForestClassifier

model2 = RandomForestClassifier(n_estimators = 280)

model2.fit(train_x2, train_y2)

RandomForestClassifier(n_estimators=280)

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, roc_curve


y_pred2 = model2.predict(test_x2)
y_pred_proba2 = model2.predict_proba(test_x2)
roc_auc2 = roc_auc_score(test_y2, y_pred2)

print("정확도:%.2f"% accuracy_score(test_y2, y_pred2))
print("정밀도:%.3f"% precision_score(test_y2, y_pred2, average = 'weighted'))
print("재현율:%.3f"% recall_score(test_y2, y_pred2, average = 'weighted'))
print("F1:%.3f"% f1_score(test_y2, y_pred2, average = 'weighted'))
print("ROC-AUC Score : {}".format(roc_auc2))

정확도:0.90
정밀도:0.899
재현율:0.897
F1:0.893
ROC-AUC Score : 0.8533368187534441


# 다중 분류 모델

### 

### 공백으로 정규화 된 icls_cnts 를 CountVectorizer 활용 / 학습-테스트 데이터 분할

In [17]:
# 분석 변수
news = list(data['ko_text'])

# 타겟 변수
y = np.array(data['label'])

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = 'word', max_features = 7000)

news = vectorizer.fit_transform(news)

In [19]:
from sklearn.model_selection import train_test_split

Test_size = 0.1
Random_seed = 34

train_x, test_x, train_y, test_y = train_test_split(news, y, test_size = Test_size, random_state = Random_seed)

In [20]:
train_y

array([ 0.,  1.,  0., ...,  0., -1.,  0.])

### 

### RandomForest 분류 모델 사용하여 모델 구축(다중 분류)

In [21]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 280)

forest.fit(train_x, train_y)

RandomForestClassifier(n_estimators=280)

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, roc_curve


y_pred = forest.predict(test_x)
y_pred_proba = forest.predict_proba(test_x)
roc_auc = roc_auc_score(test_y, y_pred_proba, multi_class = 'ovr')

print("정확도:%.2f"% accuracy_score(test_y, y_pred))
print("정밀도:%.3f"% precision_score(test_y,y_pred, average = 'weighted'))
print("재현율:%.3f"% recall_score(test_y, y_pred, average = 'weighted'))
print("F1:%.3f"% f1_score(test_y,y_pred, average = 'weighted'))
print("ROC-AUC Score : {}".format(roc_auc))

정확도:0.79
정밀도:0.810
재현율:0.790
F1:0.771
ROC-AUC Score : 0.8966634042064628


### 최적 파라미터 조정(무의미)

from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators' : [10, 100, 300],
    'max_depth' : [0, 1, 2, 3, 4, 6, 8, 10, 12],
    'min_samples_leaf' : [1, 2, 3, 4, 5, 6, 7, 8, 12, 16],
    'min_samples_split' : [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
}

grid_cv = GridSearchCV(forest, param_grid = params, cv = 2, n_jobs = -1)
grid_cv.fit(train_x, train_y)

print(f"Best Param : {grid_cv.best_params_}")
max_depth = grid_cv.best_params_['max_depth']
n_estimators = grid_cv.best_params_['n_estimators']
min_samples_leaf = grid_cv.best_params_['min_samples_leaf']
min_samples_split = grid_cv.best_params_['min_samples_split']

RF = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf = min_samples_leaf, min_samples_split = min_samples_split)
RF.fit(train_x, train_y)

y_pred = RF.predict(test_x)
y_pred_probability = RF.predict_proba(test_x)

print("정확도:%.2f"% accuracy_score(test_y, y_pred))
print("정밀도:%.3f"% precision_score(test_y,y_pred, average = 'weighted'))
print("재현율:%.3f"% recall_score(test_y, y_pred, average = 'weighted'))
print("F1:%.3f"% f1_score(test_y,y_pred, average = 'weighted'))

# 모델 적용

In [23]:
real_data = pd.read_csv('뉴스목록2022-11-01_2022-11-09.csv', encoding = 'cp949')
real_data['ko_text'] = real_data['icls_cnts'].apply(lambda x: text_cleaning(x))

In [24]:
real_news1_1 = real_data['ko_text']
real_news1 = list(real_data['ko_text'])
real_news1 = vectorizer.fit_transform(real_news1)

In [25]:
result1 = model1.predict(real_news1)
result1 = pd.Series(result1)

1차 데이터프레임 구축

In [26]:
real_news_first = pd.concat([real_news1_1, result1], axis = 1)
real_news_first.columns = ['ko_text', 'label']

In [27]:
trash1 = real_news_first['label'].isin([0])
real_news_second = real_news_first[~trash1]
# 1차 데이터 프레임
real_news_second.reset_index(drop = True, inplace = True)

In [28]:
real_news2_1 = real_news_second['ko_text']
real_news2 = list(real_news_second['ko_text'])
real_news2 = vectorizer.fit_transform(real_news2)

In [29]:
result2 = model2.predict(real_news2)
result2 = pd.Series(result2)
real_news_final = pd.concat([real_news2_1, result2], axis = 1)
real_news_final.columns = ['ko_text', 'label']
real_news_final

,ko_text,label
0,신고 늑장 대처 경찰 질타 한 점 의혹없이 진상 밝히라 윤석열 대통령이 이태원 참...,1.0
1,늑장대처 경찰 질타 행안부장관 경찰 청장 서울시장 줄줄이 사과 윤석열 대통령은 이...,1.0
2,사고 발생 시간 전에 이미 사고 현장에서 압사를 우려하면서 경찰 의 현장 통제를 ...,1.0
3,경찰 질타 왜 시간 동안 물끄러미 쳐다만 봤는가 윤석열 대통령은 일 경찰 에 있다...,1.0
4,수사 상황 지켜볼 것 이태원 핼로윈 참사 당일 경찰 의 부실 대응을 놓고 정부 책...,1.0
...,...,...
2052,대낮 음주 사고 낸 경찰 간부 중징계 처분 아시아경제 호남취재본부 박진형 기자 대낮...,-1.0
2053,경찰 소 특히 행사가 끝난 직후 사고 예방 조치가 소홀했다는 지적이 제기됐습니다 ...,1.0
2054,경찰 소 특히 행사가 끝난 직후 사고 예방 조치가 소홀했다는 지적이 제기됐습니다 ...,1.0
2055,윤희근 경찰 청장은 경찰 의 대응에 미흡한 점이 있었다며 강도 높은 감찰을 예고했습...,1.0


In [30]:
real_news_final.to_csv('a.csv')